# OPENPYXL

- **Installing ```openpyxl```**
    - ```python
        pip install openpyxl
        ```
- **Import and Load Workbook**
    - ```python
        from openpyxl import load_workbook

        wb = load_workbook('example.xlsx') # Load existing Excel file
        sheet = wb.active # Get the active sheet (usually the first)
        ```
- **Create a New Workbook**
    - ```python
        from openpyxl import Workbook

        wb = Workbook()
        sheet = wb.active
        sheet.title = "MySheet"
        ```
- **Reading Excel Data**
    - Read a specific cell:
        - ```python
            value = sheet['A1'].value
            print(value)
            ```
        - If the cell is a formula, ```cell.value``` returns the formula as a string and not the calculated result of the formula!
    - or using row/column index (1-based)
        - ```python
            value = sheet.cell(row=1, column=1).value # Returns A1


- **Loop through Rows**
    - ```python
        sheet.iter_rows(min_row=1, max_row=None, min_col=1, max_col=None, values_only=False)
        ```
        - min_row: First row to include (default is 1)
        - max_row: Last row to include (default: last non-empty row)
        - min_col: First column to include (default is 1 (Column A))
        - max_col: Last column to include (default: last non-empty column)
        - values_only: If True, returns just values, not Cell objects (default: False)
            - Cell objects give us access to:
                - ```.value```: the actual value
                - ```.row```, ```.column```, ```.coordinate```: position info
                - ```.data_type```: 's' (string), 'n' (number), 'f' (formula)
                - ```.font```, ```.fill```, ```.alignment```, etc.: formatting properties
                - ```.has_formula```: whether the cell contains formula
        - Returns a generator of tuples. Each tuple represents a row, and contains either Cell objects or raw values (if ```values_only=True```).

    - ```python
        for row in sheet.iter_rows(min_row = 2, max_row = 4, min_col = 1, max_col = 3):
            for cell in row:
                print(cell.value) # Prints values from A2 to C4.
        ```
    
    - ```python
        for row in sheet_iter_rows(min_row = 2, values_only=True):
            print(row)
        ```
        - Each ```row``` here is a tuple of cell values
            ("Alice", 30, "Canada")
            ("Bob", 25, "USA")

- **Loop through Rows/Columns with Conditions**
    - ```python
        for row in sheet.iter_rows(min_row = 2, max_row = sheet.max_row, min_col = 1, max_col = 3):
            if row[1].value == "Canada": # Column B in the current row
                print(f"Name: {row[0].value}, Country: {row[1].value}") # Only prints rows where Column B = "Canada"
        ```
    - ```python
        for col in sheet.iter_cols(min_row=2, max_row=sheet.max_row, min_col=1, max_col=3):
            for cell in col: # Iterate over each column
                if isinstance(cell.value, int) and cell.value > 100:
                    print(f"Large value found: {cell.value} at {cell.coordinate}")
        ```
    - ```python
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
            c_value = row[2].value  # Column C (index 2)
            if c_value and c_value > 100:
                print(f"{row[0].value}, {row[1].value}, {c_value}")
        ```

- **Writing Data**
    - Basic value assignment:
        - ```python
            sheet['B2'] = 'Hello'
            ```
        - ```python
            sheet.cell(row=2,column=2).value = 'Hello'
            ```
    - Writing numbers, strings, and dates:
        - ```python
            from datetime import datetime
            sheet['B3'] = 123
            sheet['B4'] = 'Data'
            sheet['B5'] = datetime.now()
            ```
    
    - Writing formulas
        - ```python
            sheet["C2"] = "=A2+B2"
            ```
            - This writes the formula as a string. Excel will evaluate it when you open the file. Note: ```openpyxl``` does not calculate formulas - only Excel does.

    - Writing with ```.cell()``` method
        - Use this for dynamic positioning:
            - ```python
                for row in range(2,10):
                    sheet.cell(row=row, column=3).value = f"=A{row}+B{row}"
                ```

    - Writing styles (Bold, Font, Fill, etc.)
        - ```python
            from openpyxl.styles import Font, PatternFill

            cell = sheet['A1']
            cell.value = "Styled Text"
            cell.font = Font(bold=True, color='FF0000') # red, bold
            cell.fill = PatternFill(fill_type = 'solid', fgColor = 'FFF00') # yellow background
            ```
    
    - Writing multiple cells at once
        - ```python
            data = [
                ['Name', 'Age', 'Country'],
                ['Eric', 33, 'Canada'],
                ['Leah', 37, 'Canada']
            ]
            for row in data:
                sheet.append(row)
            ```
        - ```.append()``` adds each sublist as a new row at the first empty row (first FULL empty row **(all columns are empty)**). Each item in the sublist goes into the next column, starting from column A
        - If your sheet already has content, ```.append()``` continues after the last used row - it doesn't overwrite. 
        - It takes an **iterable argument** such as a list or tuple.
    
    - Overwriting cell contents
        - You can directly assign a new value:
            - ```python
                sheet['A1'] = 'Old'
                sheet['A2'] = 'New' # Overwrites old. No need to clear - it just replaces the value

    - Save you work
        - ```python
            wb.save('your_file.xlsx')
            ```


- **Common Tasks**
    - Get max row/column
        - ```python
            print(sheet.max_row) # Returns the index (number) of the last non-empty row in the worksheet. It checks all columns to find the bottom-most row with any content.
            print(sheet.max_column) # Returns the index of the last non-empty column. It checks all rows to find the rightmost column that has any content.
            ```
        - These methods do not guarantee that all cells up to that row/column contain data. If you delete content from the middle of the sheet, max_row and max_column still reflect the furthest extent of what was ever written.

    - Insert or delete rows/columns
        - ```python
            sheet.insert_rows(2)     # Insert row at position 2
            sheet.delete_rows(3)     # Delete 3rd row
            sheet.insert_cols(1)     # Insert column at position A
            sheet.delete_cols(2)     # Delete column B
            ```
    - Dragging a formula down
        - Write a formula in one cell (e.g., =A2+B2 in C2).
        - Copy or "drag" that formula down the column C to match the number of rows with data in columns A and B.
        - ```openpyxl``` does not evaluate formulas, but it can copy and write them as strings.
        - You need to dynamically adjust the row number in the formula as you go down.
        - ```python
            from openpyxl import Workbook

            # Create workbook and worksheet
            wb = Workbook()
            ws = wb.active
            ws.title = "Formulas"

            # Sample data
            ws.append(["A", "B", "Sum"])  # Header
            for i in range(1, 11):
                ws.append([i, i * 10])  # Fill columns A and B

            # Drag formula down in column C (starting from row 2)
            for row in range(2, ws.max_row + 1):
                formula = f"=A{row}+B{row}"
                ws[f"C{row}"] = formula

            # Save the workbook
            wb.save("dragged_formulas.xlsx")
            ```
    - Finding the last row of a specific column
        - To find the last row with data based only on a specific column, you can loop through that column in reverse to find the last non-empty cell.
        - ```python
            for row in range(sheet.max_row, 0, -1):
                if sheet.cell(row=row, column=3) is not None: # column=3 for Column C
                    last_row_c = row
                    break
            ```

